In [1]:
import pandas as pd
from collections import defaultdict
import imaplib
import email
from email.header import decode_header
import os
from dotenv import load_dotenv
import re
from bs4 import BeautifulSoup

In [2]:
load_dotenv()
imap_server = os.getenv("IMAP_SERVER")
email_address = os.getenv("EMAIL_ADDRESS")
password = os.getenv("EMAIL_PASSWORD")

filters = [
    {"sender": "noreply.tsekk@maxima.ee", "subject": "Sinu ostutšekk!"},
    {"sender": "noreply@rimibaltic.com", "subject": "Sinu ostutšekk"},
    {"sender": "estonia-food@bolt.eu", "subject": "Delivery from Bolt Food"}
]

imap_connection = imaplib.IMAP4_SSL(imap_server)
imap_connection.login(email_address, password)

('OK', [b'pavlovs.ilja@gmail.com authenticated (Success)'])

In [17]:
def get_email_content(email_message):
    if email_message.is_multipart():
        for part in email_message.walk():
            print(part.get_content_type())
            if part.get_content_type() in ["text/plain", "text/html","multipart/mixed","multipart/related"]:
                return part.get_payload(decode=True).decode(errors='replace')
    else:
        return email_message.get_payload(decode=True).decode(errors='replace')


In [4]:
imap_connection.select("INBOX")
filtered_emails = defaultdict(list)

filter_criteria = filters[1] # rimi

sender = filter_criteria["sender"]
subject = filter_criteria["subject"]

_, message_numbers = imap_connection.search(None, f'FROM "{sender}"')

In [18]:
num = message_numbers[0].split()[1]

_, msg_data = imap_connection.fetch(num, '(RFC822)')
email_body = msg_data[0][1]
email_message = email.message_from_bytes(email_body)

email_subject, encoding = decode_header(email_message["Subject"])[0]

if isinstance(email_subject, bytes):
    email_subject = email_subject.decode(encoding or "utf-8")

if email_subject == subject:
    email_sender = email_message["From"]
    email_date = email_message["Date"]
    
    content = get_email_content(email_message)
    
    filtered_emails[sender].append({
        "subject": email_subject,
        "sender": email_sender,
        "date": email_date,
        "content": content
    })

multipart/mixed
multipart/related


AttributeError: 'NoneType' object has no attribute 'decode'

In [12]:
content

'<html>\r\n\r\n<head>\r\n    <meta http-equiv="Content-Type" content="text/html; charset=UTF-8" />\r\n</head>\r\n\r\n<table style="width:100%;font-family:Arial,sans-serif;font-weight:normal;line-height:0">\r\n    <tbody>\r\n    <tr>\r\n        <td style="background-color:white" align="center" valign="top">\r\n<table style="margin:0;padding:0;width:700px;line-height:normal" cellspacing="0" cellpadding="0" border="0" width="700">\r\n    <tbody><tr>\r\n        <td style="line-height:0;padding:60px 0 0 0;line-height:0" valign="top">\r\n            <img src="cid:bgImage" style="max-width:100%;line-height:0">\r\n        </td>\r\n    </tr>\r\n     </tbody>\r\n</table>\r\n\r\n<table style="margin:0;padding:0;width:770px" width="700" cellspacing="0" cellpadding="0" border="0">\r\n    <tbody>\r\n    <tr>\r\n        <td  style="background-color:white;padding:20px 0px 0 60px">\r\n             <h2 style="color:#363d40;font-size:24px;line-height:28px">Hea klient,</h2>\r\n             <br>\r\n       

In [ ]:
imap_connection.logout()